# IE2 and RK4 for IVP

**(a)** Write a program to solve a system of differential equations using the IE2 method.

**(b)** Write a program to solve a system of differential equations using the RK4 method.

**(c)** Test your implementation for the IE2 and RK4 methods on the IVP

$u' = -2t u$, $\ 0 \le t \le 2$, $\ u(0) = 2$; \
$\hat{u}(t) = 2e^{-t^2}$

Solve with $n=30,60,90,\ldots,300$ and plot the max-error $||u-\hat{u}||_\infty=\max_{0\leq i\leq n} |u_i-\hat{u}(t_i)|$ as a function of the number of function evaluations in a log-log plot, together with a line showing the expected rate of convergence.

I actually show the fit instead of the expected rate of convergence, but I show the slope of the fit

In [ ]:
using Plots
using Markdown
using LaTeXStrings
using Printf
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\linear_systems.jl")
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\diff_equations.jl")
function autosave(fig, filename)
    path = "C:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\relazione\\immagini"
    savefig(fig, joinpath(path, filename))
end

In [ ]:
n = 30:30:300
a, b = 0, 2
function f(t::Number, u::Number)
    return -2.0 * t * u
end
F = [f]

u0 = 2
U0 = [u0]
u_exact = t -> 2*exp(-t^2)

colors = ["#0F2080",  # blu scuro
          "#F5793A",  # arancione
          "#A95AA1",  # viola
          "#85C0F9"]  # azzurro

In [ ]:
#Error section
err_inf2 = Float64[]
err_inf4 = Float64[]
for nk in n
    t = range(a, b; length=nk+1)
    u_ex = u_exact.(t)
    u_num2 = IE2(F, U0, nk, a, b)[1]
    u_num4 = RK4(F, U0, nk, a, b)[1]
    push!(err_inf2, maximum(abs.(u_num2 .- u_ex)))
    push!(err_inf4, maximum(abs.(u_num4 .- u_ex)))
end

Puoi anche sovrapporli invece che metterli diversi, attento solo al numero di valutazioni delle funzioni. I punti 
dovrebbero incolonnarsi uno ogni due

In [ ]:
#Fit section
D = hcat(ones(length(n)), log.(n))
c2 = least_sq(D, log.(err_inf2))
c4 = least_sq(D, log.(err_inf4))

x_ticks0 = [60]
x_ticks1 = [i for i in 120:120:600]
x_ticks2 = [i for i in 840:240:1200]
x_ticks = vcat(x_ticks0, x_ticks1, x_ticks2)
#Plot section
fig1 = plot(figsize=(800, 600),
           xlabel=L"Function\ evaluations\ ", ylabel=L"Error",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           #yrange = (1e-6, 2e-2),
           xscale = :log10, yscale = :log10,
           xticks = x_ticks, yticks = [10.0^i for i in 0:-1:-16],
           #xticks = [10^i for i in 0:0.2:4], yticks = [10.0^i for i in 0:-1:-16],
           xformatter = x -> @sprintf("%.0f", x), yformatter = y -> @sprintf("%.0e", y),
           legend = :bottomleft,
           legendfontsize = 10
          )

plot!(fig1, 2 .* n, err_inf2,
      label = L"IE2\ ||u_i-\hat{u_i}\ ||_\infty",
      color= colors[1],
      marker = :circle,
      lw = 4.0,
     )

plot!(fig1, 2 .* n, exp(c2[1]) .* n .^ c2[2],
    label = latexstring("Fit slope=", @sprintf("%.3f", c2[2]), ")"),
    color = colors[2],
    linestyle = :dash,
    lw = 2.0,
     )

plot!(fig1, 4 .* n, err_inf4,
      label = L"RK4\ ||u_i-\hat{u_i}\ ||_\infty",
      color= colors[3],
      marker = :circle,
      lw = 4.0,
     )
plot!(fig1, 4 .* n, exp(c4[1]) .* n .^ c4[2],
    label = latexstring("Fit slope=", @sprintf("%.3f", c4[2]), ")"),
    color = colors[4],
    linestyle = :dash,
    lw = 2.0,
     )

autosave(fig1, "6311.pdf")
display(fig1)